# Gestión Moderna de Portafolio
### Autores Bernardo León y Carlos Zapata
### (C) Copyright 2023

## Capitulo 6: Medida Omega $\Omega$


In [5]:
%pip install --quiet yfinance

In [6]:
#Librerías usadas
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")

### Ejemplo 6.1

In [58]:
# Información histórica para las acciones: AAPL, AMZN, GOOG y MSFT
fechai = "2009-12-01"
fechaf = "2021-12-31"
periodicidad = "1Mo"
activos = ["AAPL","AMZN","GOOGL","MSFT"]
precios = yf.download(activos,start=fechai,end=fechaf,interval=periodicidad)['Adj Close'].dropna()
retornos = np.log(precios/precios.shift(1)).dropna()
mu = retornos.mean()*12
cov = retornos.cov()*12
var = np.diag(cov)
sigma = np.sqrt(var)
n = len(mu)

[*********************100%***********************]  4 of 4 completed


In [59]:
# Calculo excesos y umbral h
h = 0
excesos = retornos-h
pos_ret = excesos[excesos>h].sum()
neg_ret = excesos[excesos<h].sum()
omegai = pos_ret / (-neg_ret )
omegai

AAPL     2.100586
AMZN     2.040308
GOOGL    1.787564
MSFT     2.168691
dtype: float64

# Optimización usando SCIPY

In [60]:
from scipy.optimize import minimize

def omega(pesos, retornos, h):
  port_return = retornos @ pesos
  excesos = port_return-h
  pos_ret = excesos[excesos>h].sum()
  neg_ret = excesos[excesos<h].sum()
  return pos_ret / (-neg_ret )

#
def optimize_omega(retornos, h):
  pesos = np.repeat(1/n,n)
  objective = lambda pesos: -omega(pesos, retornos, h)
  bounds = [(0, 1)] * n
  solution = minimize(objective, pesos, bounds=bounds)
  return solution.x/sum(solution.x)

h= 0
wpo = optimize_omega(retornos, h)
wpo

array([0.18140004, 0.36459492, 0.02634756, 0.42765748])